In [4]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import face_recognition
import joblib
import openpyxl

def load_images(dataset_path):
    X = []
    y = []

    for person_name in os.listdir(dataset_path):
        person_dir = os.path.join(dataset_path, person_name)
        if os.path.isdir(person_dir):  # Check if it's a directory
            for image_name in os.listdir(person_dir):
                image_path = os.path.join(person_dir, image_name)
                if os.path.isfile(image_path):  # Check if it's a file
                    try:
                        image = face_recognition.load_image_file(image_path)
                        encoding = face_recognition.face_encodings(image)
                        if len(encoding) > 0:
                            X.append(encoding[0])
                            y.append(person_name)
                    except Exception as e:
                        print(f"Error processing image {image_path}: {e}")

    return np.array(X), np.array(y)

def train_and_save_model(X_train, y_train, model_path):
    svm = SVC(kernel='linear')  # You can change the kernel as per your requirement
    svm.fit(X_train, y_train)
    joblib.dump(svm, model_path)
    return model_path

def update_excel_trained(model_path, success):
    workbook = openpyxl.load_workbook('students.xlsx')
    sheet = workbook.active
    sheet['D1'] = "Trained"
    
    if success:
        trained_status = "✔"
    else:
        trained_status = "✘"
    
    for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=1):
        for cell in row:
            if cell.value == os.path.splitext(os.path.basename(model_path))[0]:
                cell.offset(column=3).value = trained_status
    
    workbook.save('students.xlsx')

def main():
    dataset_path = "datasets"
    X, y = load_images(dataset_path)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_path = train_and_save_model(X_train, y_train, "students_trained_model.pkl")
    
    if os.path.exists(model_path):
        update_excel_trained(model_path, True)
        print(f"Model trained and saved at {model_path}. Tick added in 'Trained' column.")
    else:
        update_excel_trained(model_path, False)
        print(f"Model training failed. Cross added in 'Trained' column.")

if __name__ == "__main__":
    main()


Model trained and saved at students_trained_model.pkl. Tick added in 'Trained' column.
